# Data scraping and analysis of the recent trending status of the fast food chain Tropical Hut

snscraper has a very easy to use python wrapper than can be used to scrape tweets from a public Twitter account.

In [2]:
from tqdm import tqdm
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import date, timedelta
from scrape_utils import scrape_twitter

In [3]:
scrape_twitter(query="tropical hut", since="2022-01-01")

tropical hut since:2022-01-01 until:2022-06-20 


8032it [06:57, 19.26it/s]


In [5]:
df = pd.read_csv("tweets_tropical hut.csv")
df.sort_values("likes", ascending=False)

,tweet_id,date,tweet,retweets,likes,quote_tweets,replies,retweeted_tweet,quoted_tweet,user_name,user_display_name,user_description,user_verified,user_location,user_created,user_followers,user_following,user_tweets_count
1994,1537658649580933120,2022-06-17 04:49:25+00:00,To those who ordered from the Tropical Hut Mak...,2668,35921,1053,146,NaN,NaN,tammydavid,Tammy David,Strategy • Sports • Storytelling vitaminsiga@g...,False,"📍 MNL, Philippines",2009-01-17 03:56:23+00:00,2946,3995,38089
7331,1535914548791693313,2022-06-12 09:18:59+00:00,"Tropical Hut in Escolta, ako lang ang customer...",3526,34312,3596,530,NaN,NaN,dumidyeypee,JP ,Stocks | Numbers | Finance | Data | Politics |...,False,NaN,2013-01-27 10:52:27+00:00,8582,3063,162652
3206,1537298789643726848,2022-06-16 04:59:27+00:00,We jumped into the trend! \nOrdered a lot from...,1650,24774,1718,188,NaN,NaN,MINlRM,Penguiné,Environmental Biology student ...,False,NaN,2017-12-02 07:34:58+00:00,1436,1823,38273
4666,1536947672875008000,2022-06-15 05:44:14+00:00,Ordered a Tropical Hut burger and a Jollibee b...,1916,18302,1550,208,NaN,NaN,LiaSanMateo,Lia San Mateo,NaN,False,MNL,2009-04-27 05:54:11+00:00,889,403,17994
428,1538134434347888640,2022-06-18 12:20:00+00:00,I now fully understand why this hype means so ...,854,11203,275,102,NaN,NaN,dumidyeypee,JP ,Stocks | Numbers | Finance | Data | Politics |...,False,NaN,2013-01-27 10:52:27+00:00,8581,3063,162652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3710,1537153247093620736,2022-06-15 19:21:07+00:00,Matry mag grab mamya\n#Tropical Hut https://t....,0,0,0,0,NaN,https://twitter.com/dumidyeypee/status/1535918...,CReviewee,Caguiwow_Reviewee,Caguiwow_reviewee\nTruth and Justice shall pre...,False,NaN,2022-04-27 18:29:14+00:00,212,286,323
3705,1537157289375039488,2022-06-15 19:37:11+00:00,miss ko na yung Tropical Hut Spaghetti 🥰 https...,0,0,0,0,NaN,https://twitter.com/AteMongPoz/status/15370350...,dhee_star,DheeStar Channel,THINK POSITIVE 💪🙏❤️. \nBroke Multifandom Fan 😅...,False,NaN,2020-12-23 05:05:41+00:00,75,550,1440
3704,1537157647400464385,2022-06-15 19:38:36+00:00,"Honestly, Tropical Hut forever pa din ako 😋 ht...",0,0,0,0,NaN,https://twitter.com/LiaSanMateo/status/1536947...,MissOdessa31,𝐃𝐞𝐭𝐭𝐞™,Dashboard and Profile Study,False,"Quezon City, Philippines",2010-09-21 16:26:06+00:00,331,178,24229
3703,1537158503822872576,2022-06-15 19:42:00+00:00,The best talaga for me ang Tropical Hut 💚🌴,0,0,0,0,NaN,NaN,cespanganiban,𝗖𝗲𝘀 ,𝖫𝖺𝗎𝗀𝗁𝗌 𝗐𝗂𝗍𝗁𝗈𝗎𝗍 𝖿𝖾𝖺𝗋 𝗈𝖿 𝗍𝗁𝖾 𝖿𝗎𝗍𝗎𝗋𝖾. 🌻,False,Republika Ng Pilipinas,2009-08-30 15:01:58+00:00,484,138,80002
